In [6]:
import os
import pandas as pd

# ======= CONFIGURAÇÕES DE TODAS AS MODALIDADES (CAMINHOS CORRIGIDOS) =======
modalidades = [
    {
        'nome': 'audio_balanced',
        'master': '../metadata/master_metadata_audio_balanced.csv',
        'data_subfolder': '../data/audio/audio_balanced',
        'file_ext': '.mp3',
        'type': 'audio'
    },
    {
        'nome': 'audio_complete',
        'master': '../metadata/master_metadata_audio_complete.csv',
        'data_subfolder': '../data/audio/audio_complete',
        'file_ext': '.mp3',
        'type': 'audio'
    },
    {
        'nome': 'lyrics_balanced',
        'master': '../metadata/master_metadata_lyrics_balanced.csv',
        'data_subfolder': '../data/lyrics/lyrics_balanced',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'lyrics_complete',
        'master': '../metadata/master_metadata_lyrics_complete.csv',
        'data_subfolder': '../data/lyrics/lyrics_complete',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'bimodal_balanced_audio',
        'master': '../metadata/master_metadata_bimodal_balanced.csv', 
        'data_subfolder': '../data/bimodal/bimodal_balanced/audio',
        'file_ext': '.mp3',
        'type': 'audio'
    },
    {
        'nome': 'bimodal_balanced_lyrics',
        'master': '../metadata/master_metadata_bimodal_balanced.csv', 
        'data_subfolder': '../data/bimodal/bimodal_balanced/lyrics',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'bimodal_complete_audio',
        'master': '../metadata/master_metadata_bimodal_complete.csv',
        'data_subfolder': '../data/bimodal/bimodal_complete/audio',
        'file_ext': '.mp3',
        'type': 'audio'
    },
    {
        'nome': 'bimodal_complete_lyrics',
        'master': '../metadata/master_metadata_bimodal_complete.csv',
        'data_subfolder': '../data/bimodal/bimodal_complete/lyrics',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
]

# ======= FUNÇÃO DE CHECAGEM =======
def checar_modalidade(nome, master_csv, data_subfolder, file_ext, tipo):
    missing = []
    try:
        df = pd.read_csv(master_csv)
    except Exception as e:
        print(f"ERRO ao ler {master_csv}: {e}")
        return None

    print(f"\n[{nome}] Total de músicas no master: {len(df)}")

    # Padroniza nomes de colunas para minúsculas para robustez
    df.columns = [c.lower() for c in df.columns]

    for idx, row in df.iterrows():
        quad = row.get('quadrant')
        if quad is None:
            print(f"[{nome}] Coluna 'quadrant' não encontrada na linha {idx}!")
            continue

        song_id = row['song_id']
        
        # --- INÍCIO DA CORREÇÃO ---
        # Monta o nome da pasta do quadrante de forma segura,
        # lidando com valores como 'Q4' (texto) ou 4 (número)
        if isinstance(quad, str) and quad.startswith('Q'):
            quad_folder = quad
        else:
            # Converte para float primeiro para lidar com casos como '4.0'
            quad_folder = f'Q{int(float(quad))}' 
        # --- FIM DA CORREÇÃO ---

        file_path = os.path.join(data_subfolder, quad_folder, f'{song_id}{file_ext}')
        
        if not os.path.exists(file_path):
            missing.append({'modalidade': nome, 'song_id': song_id, 'quadrant': quad, 'path_esperado': file_path})

    print(f"[{nome}] Arquivos faltando: {len(missing)}")
    return missing

# ======= EXECUTA PARA TODAS AS MODALIDADES (LÓGICA CORRIGIDA) =======
all_missing = []
houve_erro_leitura = False # <-- CORRIGIDO: Adiciona um sinalizador de erro

for m in modalidades:
    # Tenta ler apenas os ficheiros master que existem
    if not os.path.exists(m['master']):
        print(f"AVISO: Ficheiro master '{m['master']}' não encontrado, a pular esta verificação.")
        continue

    faltantes = checar_modalidade(m['nome'], m['master'], m['data_subfolder'], m['file_ext'], m['type'])
    
    if faltantes is None:
        houve_erro_leitura = True # <-- CORRIGIDO: Sinaliza que um erro ocorreu
    else:
        all_missing.extend(faltantes)

# ======= SALVA RELATÓRIO FINAL (LÓGICA CORRIGIDA) =======
print("\n--- Relatório Final da Validação ---")
if houve_erro_leitura:
    print("❌ A verificação não pôde ser concluída devido a erros na leitura de um ou mais ficheiros de metadados.")
else:
    if all_missing:
        df_missing = pd.DataFrame(all_missing)
        df_missing.to_csv('relatorio_arquivos_faltantes.csv', index=False)
        print(f"⚠️ Encontrados {len(all_missing)} ficheiros em falta. Relatório salvo em 'relatorio_arquivos_faltantes.csv'")
    else:
        print("✅ SUCESSO! Nenhum arquivo de dados faltando em nenhuma modalidade!")



[audio_balanced] Total de músicas no master: 3232
[audio_balanced] Arquivos faltando: 0

[audio_complete] Total de músicas no master: 3554
[audio_complete] Arquivos faltando: 0

[lyrics_balanced] Total de músicas no master: 2400
[lyrics_balanced] Arquivos faltando: 0

[lyrics_complete] Total de músicas no master: 2568
[lyrics_complete] Arquivos faltando: 0

[bimodal_balanced_audio] Total de músicas no master: 2000
[bimodal_balanced_audio] Arquivos faltando: 0

[bimodal_balanced_lyrics] Total de músicas no master: 2000
[bimodal_balanced_lyrics] Arquivos faltando: 86

[bimodal_complete_audio] Total de músicas no master: 2216
[bimodal_complete_audio] Arquivos faltando: 0

[bimodal_complete_lyrics] Total de músicas no master: 2216
[bimodal_complete_lyrics] Arquivos faltando: 133

--- Relatório Final da Validação ---
⚠️ Encontrados 219 ficheiros em falta. Relatório salvo em 'relatorio_arquivos_faltantes.csv'
